## 🔨 Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response. LLM does not call the tool its python we write or abstruct calls the tool/the python function. 

In this notebook, we have implemented tool without any framwork like Langchain or OpenAI SDK.
We have explained here, how you can implement tool and how it works behind the sceen

### 🔨 Example 1

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


c:\PRANAB\CODE\OpenAI\LLMEngineering\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Initialization. I am using the OpenAI API key from the .env file and the model name from the environment variable.

load_dotenv(override=True)
MODEL_NAME = os.getenv("MODEL_NAME")
openai = OpenAI()

In [4]:
system_message = """
You are a helpful assistant for an Airline called GoAI. Give short, courteous answers, no more than 1 sentence. Always be accurate. 
If you don't know the answer, say so.
"""

In [5]:
# This is a simple function that returns the price of a ticket to a destination city.
# This function can be replaced with Real API call to get the price of a ticket in production environment.


ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499", "kolkata": "$1990", "mumbai": "$1299", "delhi": "$1199", "chennai": "$1399", "hyderabad": "$1499", "bengaluru": "$1599"}

def get_ticket_price(destination_city):
    print(f"Tool called for city {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "Unknown ticket price")
    return f"The price of a ticket to {destination_city} is {price}"


In [6]:
# There's a particular dictionary structure that's required to describe our function(get_ticket_price):

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [7]:
# This is the function that will be called by the LLM for booking the ticket.

def book_ticket(destination_city:str, travel_date:str):
    print(f"Tool called | Book ticket | for city {destination_city} and travel date {travel_date}")
    if destination_city == None or travel_date == None: # This is to handle the case where the LLM does not provide the required arguments. 
        return "Please provide the destination city and travel date"
    return f"Ticket booked successfully for {destination_city} on {travel_date}"


In [8]:
# There's a particular dictionary structure that's required to describe our function(book_ticket):

booking_function = {
    "name": "book_ticket",
    "description": "This method is used to book a ticket to the destination city on the given travel date.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "travel_date": {
                "type": "string",
                "description": "The departure date for the outbound flight, in 'YYYY-MM-DD' format."
      },
        },
        "required": ["destination_city","travel_date"],
        "additionalProperties": False
    }
}


In [9]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function},{"type": "function", "function": booking_function}]

In [10]:
# We have to write that function handle_tool_call. The LLM may need to call this tool multiple times in the same response.
# So the function should be able to handle multiple tool calls, that's why we are using a for loop.
# The retun should be in proper format, so that the LLM can understand the response.tool_call_id is very important. LLM uses this to match the response with the tool call.

def handle_tool_call(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            price_details = get_ticket_price(city)
            response = {
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            }
            responses.append(response)
        elif tool_call.function.name == "book_ticket":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            travel_date = arguments.get('travel_date')
            booking_details = book_ticket(city, travel_date)
            response = {
                "role": "tool",
                "content": booking_details,
                "tool_call_id": tool_call.id
            }
            responses.append(response)
    return responses    


In [11]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL_NAME, messages=messages, tools=tools)

    print(response.choices[0])

    # LLM can call the tool multiple times in the same response. So we need to handle multiple tool calls.
    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_call(message)
        messages.append(message) # This is the last message from the LLM, that has tool call required in it. This needs to be added to the messages list.show that LLM can map the tool call and tool call output.
        messages.extend(responses)

        print(messages)
        response = openai.chat.completions.create(model=MODEL_NAME, messages=messages, tools=tools)
        print(response.choices[0])
    
    return response.choices[0].message.content

In [12]:
# Open the chatbot window
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you with your GoAI travel plans today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))
